In [160]:
import csv, pgeocode
from collections import defaultdict

coords = {}

with open('data/Substance Use Treatment.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    for line in csv_reader:
        #Stores agency name and postal code
        name = line['AGENCY_NAME']
        address = line['ORGANIZATION_ADDRESS']
        postal = address[len(address) - 7: len(address)]
        
        nomi = pgeocode.Nominatim('ca')
        
        coords.setdefault(name, {
                'Longitude': 0,
                'Latitude': 0
        })
        
        #Sets Coordinates
        coords[name]['Longitude'] = (nomi.query_postal_code(postal).longitude)
        coords[name]['Latitude'] = (nomi.query_postal_code(postal).latitude)

#Prints Location and Coordinates
for place, value in coords.items():
    #Stores coordinates in integers
    a = coords[place]['Latitude']
    b = coords[place]['Longitude']
    
    #Prints location name and coordinates
    #print(place)
    #print(f'\t{a}, {b}')

In [161]:
import csv, re
from collections import defaultdict

#Creates dictionaries
homicide = {}
cent = {}
total = 0

with open('data/Homicide.csv', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    
    #Stores all the relevant data (location, id, coordinates)
    for line in csv_reader:
        division = line['Division']
        geo = line['geometry']
        ident = line['_id']
        
        homicide.setdefault(ident, {
            'Location': 0,
            'Latitude': 0,
            'Longitude': 0
        })
        
        #Stores which district crime happened in
        if division == 'D41' or division == 'D42' or division == 'D43':
            homicide[ident]['Location'] = "Scarbourough"
        elif division == 'D22' or division == 'D23':
            homicide[ident]['Location'] = "Etobicoke"
        elif division == 'D31' or division == 'D32' or division == 'D33':
            homicide[ident]['Location'] = "North York"
        elif division == 'D54':
            homicide[ident]['Location'] = "East York"
        elif division == 'D12' or division == 'D31':
            homicide[ident]['Location'] = "Weston"
        elif division == 'D53':
            homicide[ident]['Location'] = "Forest Hill"
        elif division == 'D54':
            homicide[ident]['Location'] = "Leaside"
        elif division == 'D13':
            homicide[ident]['Location'] = "York Township"
        elif division == 'D11':
            homicide[ident]['Location'] = "Swansea"
        #else:
            #print(division)

        #Gathers coordinates of crime
        x = re.search(r"\(([A-Za-z0-9.\-,\s]+)\)", geo)
        coordinates = x.group(1).split(", ")
        
        homicide[ident]['Latitude'] = coordinates[1]
        homicide[ident]['Longitude'] = coordinates[0]
        
#Calculates average longitude and latitude of crimes per location
for i, value in homicide.items():
    loc = homicide[i]['Location']
    cent.setdefault(loc, {
        'Name': 0,
        'Lat': 0,
        'Long': 0,
        'Count': 0,
        'Percent': 0
    }) 
    #print(center)
    cent[loc]['Name'] = loc
    cent[loc]['Lat'] += float(homicide[i]['Latitude'])
    cent[loc]['Long'] += float(homicide[i]['Longitude'])
    cent[loc]['Count'] += 1
    total += 1
        
for loc, info in cent.items():
    cent[loc]['Lat'] = cent[loc]['Lat']/cent[loc]['Count']
    cent[loc]['Long'] = cent[loc]['Long']/cent[loc]['Count']
    cent[loc]['Percent'] = cent[loc]['Count']/total * 100
    #print(cent[loc]['Name'])
    #print(cent[loc]['Lat']/cent[loc]['Count'])
    #print(cent[loc]['Long']/cent[loc]['Count'])
    print(cent[loc]['Percent'])
        
            

3.0017152658662094
22.641509433962266
3.2590051457975986
19.296740994854204
12.950257289879932
23.241852487135507
5.4030874785591765
3.516295025728988
6.689536878216123


In [162]:
from ipyleaflet import Map, Marker, basemaps, Popup, Circle
from ipywidgets import HTML

#Creates map
m = Map(center = (43.6487, -79.38544), zoom = 10, min_zoom = 1, max_zoom = 20, 
    basemap=basemaps.Stamen.Terrain)
#m = Map(center = (43.6656, -79.383), zoom = 10, min_zoom = 1, max_zoom = 20)

#Adds markers
#https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html
for place, value in coords.items():
    #Stores coordinates in integers
    a = coords[place]['Latitude']
    b = coords[place]['Longitude']
    
    #Adds all the markers
    center = (a, b)
    marker = Marker(location=center, draggable=False)
    m.add_layer(marker);
    
    #Creates message
    message = HTML()
    message.value = place
    
    #Creates popup
    popup = Popup(
        location=center,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    #Adds popup to layer and associates it with the message and marker
    marker.popup = message
    
# Adds circle markers
for loc, info in cent.items():
    circle = Circle()
    circle.location = (cent[loc]['Lat'], cent[loc]['Long'])
    circle.radius = int(cent[loc]['Percent'] * 100)
    circle.color = "steelBlue"
    circle.fill_color = "steelBlue"
    m.add_layer(circle)

#Draws Map
m

Map(center=[43.6487, -79.38544], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [163]:
L.circle({lat: cent[loc]['Lat'], lng: cent[loc]['Long']}, {
        color: 'steelblue',
        radius: 500,
        fillColor: 'steelblue',
        opacity: 0.5
    }).addTo(this.map)

NameError: name 'L' is not defined